In [43]:
#%%
import os
import numpy as np
import torch
import torch.optim as optim
from torch.utils.data import TensorDataset, ConcatDataset,random_split
import torch.nn.functional as F
import torchvision
import PIL
import matplotlib.pyplot as plt
import pandas as pd
import zipfile
from tqdm import tqdm
import math
import torch.nn as nn

In [44]:
device = torch.device('cuda')
dtype = torch.float32

In [45]:
#%%

#@title

#Parameters for the antenas
visible_region = 180
resolution = 1

num_antenna = 8
c0=3e8
fc= 76e9
lambda_var = c0/fc
d = lambda_var/(2*np.sin(np.pi/180*0.5*visible_region))

coordinate=np.array([0, 1, 4, 6, 13, 14, 17, 19])
NN_size_out = (visible_region/resolution)+1
NN_size_out =  np.array(NN_size_out,dtype='i')


#Create Input and Output for the NN
#aoa = matrix or vector
#A = matrix or vector
#max_gen_ang = Max_min allowed angle when generating own angles
#use_R=True Outputs the Covariance Matrix. If FALSE, outputs the snapshot
def create_i_o(SNR,aoa='random',A='default',n_angle=-1,Data_Size=-1,max_gen_ang=visible_region/2,
              return_angle=False,return_snapshot=False,use_R=False,return_Y=True,
              c0=c0,fc=fc,lambda_var=lambda_var,visible_region=visible_region,d=d,num_antenna=num_antenna,coordinate=coordinate,
              NN_size_out=NN_size_out):

    ############# INTERNAL FUNCTIONS ########################
    #
    ########################
    def get_snapshot():
        r = np.zeros((Data_Size,n_angle,num_antenna))+0*1j
        for i in range(num_antenna):
            r[:,:,i] = A*np.exp(1j*(2*np.pi*fc*coordinate[i]*d*np.sin(np.pi*aoa/180)+2*np.pi*np.random.random())/c0)
        #
        snapshot_org = np.sum(r[:,:,:],axis=1)
        #
        snapshot = awgn(snapshot_org)
        snapshot = np.array(snapshot)
        #
        if use_R==True:
            R = np.zeros((Data_Size,num_antenna,num_antenna))+0*1j
            for i in range(Data_Size):
                R[i,:,:] = np.outer(snapshot[i,:],snapshot[i,:])
            #
            if return_snapshot==False: #R_true, S_false
                return R
            else: #R_true, S_true
                return R,snapshot
        else: #R_false, S_true
            return snapshot
    #
    ########################
    def awgn(signal_snap): #add white gaussian noise
        power_snap = 10*np.log10(np.abs(signal_snap) ** 2)
        noise_dB = power_snap - SNR
        noise_energy = 10 ** (noise_dB / 10)
        noise_to_signal = np.random.normal(0, np.sqrt(noise_energy))
        Re_noise_to_signal= np.random.random((Data_Size,num_antenna))*np.abs(noise_to_signal)
        Im_noise_to_signal = np.sqrt((noise_to_signal ** 2) - (Re_noise_to_signal ** 2))
        signal_noise = Re_noise_to_signal*((-1)**np.random.randint(0,2,(Data_Size,num_antenna))) + \
                    1j*Im_noise_to_signal*((-1)**np.random.randint(0,2,(Data_Size,num_antenna)))

        noisy_signal = signal_snap + signal_noise
        return noisy_signal
    #
    #######################################################################
    #
    ########
    #Error#
    if n_angle > num_antenna-2:
        print('The number of sources is too big regarding the number of antennas!')
        return
    if n_angle < 0 and aoa == 'random':
        print('Inform either "aoa" or the "n_angle"')
        return
    if Data_Size < 0 and aoa == 'random':
        print('Inform either "Data_Size" or the "n_angle"')
        return
    ########
    #
    #Other Vars#
    if n_angle < 0:
        n_angle = aoa.shape[1]
    elif n_angle==0:
        n_angle=1
        A = np.zeros((Data_Size,n_angle))
        A[:,:] = 0.00000001
    #
    if max_gen_ang < 90 and max_gen_ang > 0 and max_gen_ang != visible_region/2:
        visible_region = 2*max_gen_ang
    #
    if aoa == 'random':
        aoa = np.random.randint(-visible_region/2,1+visible_region/2,Data_Size*n_angle)
        aoa = np.reshape(aoa,(Data_Size,n_angle))
    else:
        if Data_Size < 0:
            Data_Size = aoa.shape[0]
    #
    if A == 'default':
        A = np.ones((Data_Size,n_angle))
    #
    if Data_Size < 1:
        if aoa.ndim == 1:
            Data_Size = 1
        else:
            Data_Size = aoa.shape[0]
    Data_Size = np.array(Data_Size,dtype='i')
    #
    #Check SNR type:
    if (type(SNR) is list and len(SNR) == 2) or (type(SNR) is np.ndarray and SNR.size == 2):#If list or array and 2 elements
        SNR = np.array(SNR)
        SNR_init = SNR[0]
        SNR_end = SNR[1]
        SNR = np.random.randint(SNR_init,1+SNR_end,Data_Size) #creates the SNR with Data_Size elements
        SNR = np.transpose(np.tile(SNR, (num_antenna,1))) #Repeats the values in the row. SNR.shape is (data,num_antenna)
    #
    elif (type(SNR) is list and len(SNR) > 2) or (type(SNR) is np.ndarray and SNR.size > 2):#If more than 2 elements give error
        print('Too many arguments in for the SNR. 1 or 2 elements')
        return
    #
    #Error#
    if A.size != aoa.size:
        print('"A" size is different than "aoa" size')
        return
    ########
    #
    #OUTPUT X
    if use_R==True:
        if return_snapshot==False: #R_true, S_false
            R = get_snapshot()
        else: #R_true, S_true
            R,snapshot = get_snapshot()

        X = R
    #
    else: #R_false, S_true
        snapshot = get_snapshot()
        X = snapshot
    #
    #OUTPUT Y
    if return_Y==True:
        NN_idx = ang2NN(aoa)
    #
        Y = (1 + np.random.random((Data_Size,NN_size_out))) / 10000
        for i in range(Data_Size):
            sumY = np.sum(Y[i,:])
            Y[i,NN_idx[i,:]] = (1 - sumY) / n_angle
    #
    #RETURN
    if  return_angle==False and return_snapshot==False and return_Y==True:
        return X,Y
    elif return_angle==True and return_snapshot==False and return_Y==True:
        return X,Y,aoa
    elif return_angle==True and return_snapshot==True  and return_Y==True:
        return X,Y,aoa,snapshot
    elif return_angle==False and return_snapshot==True and return_Y==True:
        return X,Y,snapshot
    elif return_angle==False and return_snapshot==True and return_Y==False:
        return X,snapshot
    elif return_angle==True and return_snapshot==False and return_Y==False:
        return X,aoa
    elif return_angle==False and return_snapshot==False and return_Y==False:
        return X
    elif return_angle==True and return_snapshot==True and return_Y==False:
        return X,aoa,snapshot

In [46]:
Max_sources = 4
N_angle = 3
Data_Size = 409600

#%%
Dataset_list = []
#
SNR = [0,5,10,15,20,25,30]
Dataset_list = []
for ratio in SNR:
    x,aoa = create_i_o(ratio,n_angle=3,Data_Size=Data_Size,max_gen_ang=70,
                    return_angle=True,return_Y=False)
    aoa = torch.tensor(aoa)
    x = torch.tensor(x)
    x = torch.unsqueeze(x,-2)
    hermitian_x = torch.conj(torch.transpose(x,-2,-1))
    cov_x = torch.matmul(hermitian_x,x)
    #upperCov = torch.triu(cov_x)
    #upperCov = upperCov.view(upperCov.size(0),-1)
    #get the upper triangle of the cov_x(tensor of (4096,8,8)) A and flatten it to a tensor of (4096,32)
    upper = torch.triu(cov_x)
    non_zero_elements = torch.flatten(upper[upper != 0])
    reshaped_tensor = non_zero_elements.view(Data_Size, -1)
    real = torch.real(reshaped_tensor)
    imag = torch.imag(reshaped_tensor)
    concat = torch.cat((real,imag),1)
    DataSet_WithSource = TensorDataset(concat,aoa)
    Dataset_list.append(DataSet_WithSource)

Full_dataset = ConcatDataset(Dataset_list)


In [47]:
Train_dataset, Val_dataset = random_split(Full_dataset,[2508800,358400])

In [48]:
batch_size = 128
Train_loader = torch.utils.data.DataLoader(
       dataset=Train_dataset,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True)

Val_loader = torch.utils.data.DataLoader(
       dataset=Val_dataset,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True)

In [49]:
import torch.nn.functional as F
class MLP(nn.Module):
    def __init__(self, n_channels=72,channel1=1024,channel2=2048,channel3=4096,channel4=512, channel5=1024, channel6=2048, n_classes=14):
        super().__init__()
        #####################Encoder layers##############################
        self.encoder_1 = nn.Sequential(
            nn.Linear(n_channels, channel1),
            nn.BatchNorm1d(channel1),
            nn.ReLU(inplace = True),
        )

        self.encoder_2 = nn.Sequential(
            nn.Linear(channel1, channel2),
            nn.BatchNorm1d(channel2),
            nn.ReLU(inplace = True),
        )

        self.encoder_3 = nn.Sequential(
            nn.Linear(channel2, channel3),
            nn.BatchNorm1d(channel3),
            nn.ReLU(inplace = True),
        )
        self.encoder_4 = nn.Sequential(
            nn.Linear(channel3, channel3),
            nn.BatchNorm1d(channel3),
            nn.ReLU(inplace = True),
        )
        self.encoder_5 = nn.Sequential(
            nn.Linear(channel3, channel3),
            nn.BatchNorm1d(channel3),
            nn.ReLU(inplace = True),
        )
        self.encoder_6 = nn.Sequential(
            nn.Linear(channel3, channel2),
            nn.BatchNorm1d(channel2),
            nn.ReLU(inplace = True),
        )
        self.encoder_7 = nn.Sequential(
            nn.Linear(channel2, channel1),
            nn.BatchNorm1d(channel1),
            nn.ReLU(inplace = True),
        )
        self.regression4 = nn.Sequential(
            nn.Linear(channel1, 141),
            nn.Sigmoid(),
        )
    def forward(self,x):

        x = self.encoder_1(x)

        x = self.encoder_2(x)

        x = self.encoder_3(x)

        x = self.encoder_4(x)

        x = self.encoder_5(x)

        x = self.encoder_6(x)

        x = self.encoder_7(x)

        reg4 = self.regression4(x)

        return reg4

In [50]:
Model = MLP()
weight_decay = 1e-4
optimizer = optim.Adam(Model.parameters(), lr=0.001)
Model = Model.to(device=device)
#model = model.to(device=device)  # move the model parameters to CPU/GPU
epochs = 30

In [37]:
tst = torch.rand(128,72)
out = Model(tst)
print(out.shape)

torch.Size([128, 141])


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [51]:

def create_label(y):
  a = torch.zeros(128,141)
  a = a.to(device=device)
  y = (y+70).long()
  a.scatter_(1,y,1)
  return a

In [52]:
label = create_label(y.to(device='cpu'))
print(label[0,:])
print(y[0,:])

RuntimeError: ignored

In [ ]:
# tst = torch.rand(128,72)
# out = Model(tst)
# print(out.shape)
for e in range(epochs):
    for t, (x, y) in enumerate(tqdm(Train_loader)):
        Model.train()  # put model to training mode
        x = x.to(device=device, dtype=dtype)
        y = y.to(device=device, dtype=dtype)
        optimizer.zero_grad()
        aoa3 = Model(x)
        _,prediction = torch.topk(aoa3,k=3,dim=1)
        # prediction.requires_grad(True)
        label = create_label(y)
        loss_reg = F.mse_loss(aoa3, label)
        loss_reg.backward()

        optimizer.step()

    if e%2 == 0:
        print('Epoch %d, Classification loss = %.4f' % (e, loss_reg.item()))

    with torch.no_grad():
        Model.eval()
        for idx,(x,y) in enumerate(tqdm(Val_loader)):
                x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
                y = y.to(device=device, dtype=dtype)
                reg4 = Model(x)
                _,estimation = torch.topk(reg4,k=3,dim=1)
                directions = estimation-70
                labell = create_label(y)
                loss_reg_val = F.mse_loss(reg4, labell)
                loss = loss_reg_val
        if e%2 == 0:
            print('Epoch %d, validation loss = %.4f' % (e, loss))
            print(f'Epoch {e}, Truth ={y[0,:]}')
            print(f'Epoch {e}, sources ={directions[0,:]}')


100%|██████████| 19600/19600 [02:37<00:00, 124.36it/s]


Epoch 0, Classification loss = 0.0114


100%|██████████| 2800/2800 [00:08<00:00, 318.16it/s]


Epoch 0, validation loss = 0.0101
Epoch 0, Truth =tensor([ 60.,  22., -21.], device='cuda:0')
Epoch 0, sources =tensor([-21,  22,  23], device='cuda:0')


100%|██████████| 19600/19600 [02:37<00:00, 124.46it/s]


...

100%|██████████| 19600/19600 [02:37<00:00, 124.59it/s]


Epoch 26, Classification loss = 0.0062


100%|██████████| 2800/2800 [00:08<00:00, 318.10it/s]


Epoch 26, validation loss = 0.0068
Epoch 26, Truth =tensor([-15.,  64., -69.], device='cuda:0')
Epoch 26, sources =tensor([-15,  63, -69], device='cuda:0')


 57%|█████▋    | 1596/2800 [00:05<00:03, 321.97it/s]

In [15]:
print(y.shape)

torch.Size([128, 3])
